<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/032_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



### 🧱 Why Structured Output Matters

Before function calling, developers had to rely on **prompt engineering** to *hope* the model returned well-formed JSON. But this was:

* ✅ *Unreliable* – The model could easily miss a comma or format a string wrong
* ✅ *Fragile* – Changes in prompts or slight wording variations would break parsers
* ✅ *Error-prone* – You needed lots of custom logic to handle malformed responses

### ✅ What Function Calling Solves

Function calling changes everything by **guaranteeing structured output**, thanks to OpenAI’s API enforcing a strict format.

> *“Instead of treating function execution as a free-form text generation task, function calling APIs allow us to explicitly define the tools available to the model using JSON Schema.”*

That means:

* The **LLM doesn't try to "generate JSON"** anymore — it *selects* a function and fills in a structured argument block.
* The **OpenAI API enforces that structure** — invalid responses are rejected before they even reach you.
* You get a `tool_calls` block like this, not unstructured text:

```json
{
  "tool_calls": [
    {
      "function": {
        "name": "search_file_names",
        "arguments": "{ \"keyword\": \"memory\", \"case_sensitive\": false }"
      }
    }
  ]
}
```

No more manual extraction from text like:

> “Sure, the file names that contain ‘memory’ are: ...”

Now you just call:

```python
tool_name = tool_call.function.name
args = json.loads(tool_call.function.arguments)
```

### 🔄 You’re Free to Focus on Behavior

By delegating **format enforcement** to the API, you can focus on:

* Designing great tools
* Structuring decision logic
* Making agents more intelligent

It moves LLMs from being *generative text bots* to *structured collaborators.*


In [1]:
%pip install -qU dotenv openai

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import re
import textwrap
from typing import List
from litellm import completion

load_dotenv("/content/API_KEYS.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 1. Define the Tool Functions
#------------------------------
def list_files() -> List[str]:
    """List files in the current directory."""
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"

# First, we define the actual Python functions that will be executed.
# These contain the business logic for each tool and handle the actual operations the AI agent can perform.

# 2. Create a Function Registry
#------------------------------
tool_functions = {
    "list_files": list_files,
    "read_file": read_file
}
# We maintain a dictionary that maps function names to their corresponding Python implementations.
# This registry allows us to easily look up and execute the appropriate function when the model calls it.

# 3. Define Tool Specifications Using JSON Schema
#-------------------------------------------------
tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the directory.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {"file_name": {"type": "string"}},
                "required": ["file_name"]
            }
        }
    }
]
# This is where we describe our tools to the model. Each tool specification includes:
# A name that matches a key in our tool_functions dictionary
# A description that helps the model understand when to use this tool
# Parameters defined using JSON Schema, specifying the expected input format
# Note how the list_files function takes no parameters (empty “properties” object), while read_file requires a “file_name” string parameter. The model will use these specifications to generate properly structured calls.

# 4. Set Up the Agent’s Instructions
#------------------------------------
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

If a user asks about files, documents, or content, first list the files before reading them.
"""
}]
# The system message provides guidance on how the agent should behave.
# With function calling, we don’t need to instruct the model on how to format its outputs - we only need to focus on the decision-making logic.

# 5. Prepare the Conversation Context
#--------------------------------------
user_task = input("What would you like me to do? ")
memory = [{"role": "user", "content": user_task}]
messages = agent_rules + memory
# We combine the system instructions with the user’s input to create the conversation context.

# 6. Make the API Call with Function Definitions
#-----------------------------------------------
response = completion(
    model="openai/gpt-4o",
    messages=messages,
    tools=tools,
    max_tokens=1024
)
# The critical difference here is the inclusion of the tools parameter,
# which tells the model what functions it can call. This is what activates the function calling mechanism.

# 7. Process the Structured Response
#------------------------------------
tool = response.choices[0].message.tool_calls[0]
tool_name = tool.function.name
tool_args = json.loads(tool.function.arguments)
When using function calling, the response comes back with a dedicated tool_calls array rather than free-text output. This ensures that:

# The function name is properly identified
# The arguments are correctly formatted as valid JSON
# We don’t need to parse or extract from unstructured text
# 8. Execute the Function with the Provided Arguments
#-----------------------------------------------------
result = tool_functions[tool_name](**tool_args)
# Finally, we look up the appropriate function in our registry and call it with the arguments
# the model provided. The **tool_args syntax unpacks the JSON object into keyword arguments.


Let’s walk through this code **step by step**, and I’ll highlight what you should **focus on and learn**, especially with respect to function calling and agent design.

---

## 🧠 Conceptual Overview

This script demonstrates a **minimal working AI agent** that:

1. Accepts user input.
2. Lets the **LLM decide** whether a tool is needed.
3. **Parses the tool call** (automatically structured).
4. Executes the correct Python function with the tool arguments.
5. Returns the tool result.

This is the core of **tool-using agents**. Let’s now go line by line.

---

## 🔹 1. Basic Imports and Tool Functions

```python
import json
import os
from typing import List

from litellm import completion
```

* `json`: For parsing structured tool arguments.
* `os`: Used in `list_files()` to interact with the file system.
* `litellm`: This is a lightweight wrapper for OpenAI (used instead of `openai` package) – interchangeable for your purposes.

---

## 🔹 2. Tool Functions (The "Real" Logic)

```python
def list_files() -> List[str]:
    return os.listdir(".")
```

* This just returns all files in the current directory.
* 🧠 **Lesson**: The tool logic is your job — the LLM just *calls* it.

```python
def read_file(file_name: str) -> str:
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
```

* Basic file reading with error handling.
* 🧠 **Lesson**: Include good error messaging — the LLM can explain it to users if needed.

---

## 🔹 3. Tool Registry (Tool Router)

```python
tool_functions = {
    "list_files": list_files,
    "read_file": read_file
}
```

* A dictionary router so we can call the right function dynamically.
* 🧠 **Key Skill**: Match tool name to the function — super useful for scaling.

---

## 🔹 4. Tool Schemas (JSON Schema for the LLM)

```python
tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the directory.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    },
```

* You’re telling the LLM:

  * This tool exists (`list_files`)
  * It takes **no arguments**
  * It should return a list of filenames

```python
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {"file_name": {"type": "string"}},
                "required": ["file_name"]
            }
        }
    }
]
```

* `read_file` expects one required parameter: `file_name`
* 🧠 **Lesson**: This is the key part that tells the LLM how to call your function.

---

## 🔹 5. System Prompt (Agent Rules)

```python
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

If a user asks about files, documents, or content, first list the files before reading them.
"""
}]
```

* This sets behavior — think of it like a “job description”
* 🧠 **Tip**: You can get clever here and instruct *decision-making strategies*

---

## 🔹 6. User Input and Messages

```python
user_task = input("What would you like me to do? ")
memory = [{"role": "user", "content": user_task}]
messages = agent_rules + memory
```

* User input is combined with the system rule to form the full context for the LLM.

---

## 🔹 7. Making the Completion Call

```python
response = completion(
    model="openai/gpt-4o",
    messages=messages,
    tools=tools,
    max_tokens=1024
)
```

* 🔥 **The key part**:

  * You provide tools to the LLM
  * The LLM decides if a tool is needed and fills in structured arguments

---

## 🔹 8. Handling the Tool Call

```python
tool = response.choices[0].message.tool_calls[0]
tool_name = tool.function.name
tool_args = json.loads(tool.function.arguments)
result = tool_functions[tool_name](**tool_args)
```

* 🔍 **Take Note**:

  * No prompt parsing
  * No “guessing” what the model meant
  * You get a **guaranteed structured response** with name + arguments

---

## 🔹 9. Displaying the Result

```python
print(f"Tool Name: {tool_name}")
print(f"Tool Arguments: {tool_args}")
print(f"Result: {result}")
```

* Simple debug-style output.

---

## ✅ What You Should Learn from This

| Concept                               | Why It Matters                                  |
| ------------------------------------- | ----------------------------------------------- |
| **Tool functions**                    | You still write these — traditional coding      |
| **Tool schemas**                      | LLM needs this to use tools properly            |
| **Structured arguments**              | No more manual parsing — huge upgrade           |
| **Tool routing**                      | You need a clean way to call the right function |
| **System message design**             | Controls how the agent behaves                  |
| **LLM doesn’t “guess” output format** | OpenAI’s API enforces structure                 |




🎯 **Everything you've been learning over the past few notebooks has been building up to this moment.** Let’s connect the dots clearly:

---

### 🧱 What You’ve Been Building

| **Concept**                     | **What You Did**                                                               | **Why It Matters**                                            |
| ------------------------------- | ------------------------------------------------------------------------------ | ------------------------------------------------------------- |
| ✅ **Python functions**          | Built tools like `list_files()` and `read_file()`                              | Actual business logic to execute tasks                        |
| ✅ **JSON tool schema**          | Defined `type`, `name`, `parameters`, `description`                            | Tells the LLM *what tools exist and how to use them*          |
| ✅ **LLM prompt architecture**   | Created `generate_agent_response()` using OpenAI’s `chat.completions.create()` | Enables the LLM to pick a tool or reply naturally             |
| ✅ **Routing logic**             | Wrote `tool_router()` to dispatch calls                                        | Bridges LLM’s intent to your real Python functions            |
| ✅ **Tool call handling**        | Created `handle_tool_call()` and debug modes                                   | Executes, inspects, and optionally continues the conversation |
| ✅ **Structured output parsing** | Noted how `tool_call.function.arguments` is a stringified JSON                 | Understood the benefit of OpenAI enforcing structure          |

---

### 🔍 Why This Lecture Summary Ties It All Together

The lecture highlights how OpenAI’s **function calling APIs**:

* Remove the pain of forcing the LLM to output JSON manually
* Let you use schemas (like real APIs) to describe tool usage
* Guarantee tool call structure so you don’t need brittle parsing code
* Make the LLM your **thinking partner**, not your parser

---

### 🤯 The Big Insight

> **You're not just building an app that uses GPT. You’re building a modular AI system.**
>
> One where:
>
> * The LLM **understands the user's intent**
> * The schema **tells the LLM how to act on that intent**
> * The router **executes real-world logic**
> * The API **ensures clean boundaries and valid calls**


